In [1]:
!pip install pulp

#!sudo apt-get install coinor-cbc glpk-utils coinor-clp
!conda install -c conda-forge pulp

     |████████████████████████████████| 40.6MB 101kB/s 
/bin/bash: conda: command not found


In [2]:
import pulp

print(pulp.__version__)

2.3.1


In [3]:
from pulp import *

In [9]:
prob = LpProblem("Okna",LpMaximize)
x1=LpVariable("Okna_aluminiowe",0)
x2=LpVariable("Okna_drewniane",0)
prob += 3000*x1 + 5000*x2, "Zysk"
prob += x1 <= 4, "Fabryka_1"
prob += 2*x2 <= 12, "Fabryka_2"
prob += 3*x1 + 2*x2 <= 18, "Fabrryka_3"


prob.writeLP("Okna.lp")
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Zysk = ", value(prob.objective))

Status: Optimal
Okna_aluminiowe = 2.0
Okna_drewniane = 6.0
Zysk =  36000.0


In [30]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

style = {'description_width': 'initial'}
fabryka_1_slider = FloatSlider(min=0,max = 50, value= 4, description="fabryka_1_slider", style= style )
fabryka_2_slider = FloatSlider(min=0,max = 50,value= 12, description="fabryka_2_slider", style= style )
fabryka_3_slider = FloatSlider(min=0,max = 50, value= 18, description="fabryka_3_slider", style= style )
def hurt_produkcja(fabryka_1 = 4,
    fabryka_2 = 12,
    fabryka_3= 18,):
  
  prob = LpProblem("Okna",LpMaximize)
  x1=LpVariable("Okna_aluminiowe",0, None, LpInteger)
  x2=LpVariable("Okna_drewniane",0, None, LpInteger)
  prob += 3000*x1 + 5000*x2, "Zysk"
  prob += x1 <= fabryka_1, "Fabryka_1"
  prob += 2*x2 <= fabryka_2, "Fabryka_2"
  prob += 3*x1 + 2*x2 <= fabryka_3, "Fabrryka_3"

  prob.writeLP("Okna.lp")
  prob.solve()
  print("Status:", LpStatus[prob.status])
  for v in prob.variables():
    print(v.name, "=", v.varValue)
  print("Zysk = ", value(prob.objective))
  shadows = [{'name':name, 'shadow price': c.pi, "slack": c.slack} for name, c in prob.constraints.items()]
  print("*"*10+"Shadow Prices"+"*"*10)
  print(pd.DataFrame(shadows))
interact(hurt_produkcja,fabryka_1 = fabryka_1_slider,
    fabryka_2 = fabryka_2_slider,
    fabryka_3 = fabryka_3_slider,)

interactive(children=(FloatSlider(value=4.0, description='fabryka_1_slider', max=50.0, style=SliderStyle(descr…

<function __main__.hurt_produkcja>